In [81]:
import numpy as np 
import pandas as pd
import pickle
import xgboost as xgb
import re

In [91]:
vivino_reviews = pd.read_csv('../input/wine-preferences/data1_100.csv', dtype={'Note': str}).drop_duplicates(keep='first').dropna(subset=['Note'])
vivino_reviews.head()

,Year,Wine ID,Wine Name,Country,Structure,Style,User Rating,Note,CreatedAt,Wine Type
0,2017,1879,Malbec,Argentina,"{'acidity': 2.7203922, 'fizziness': None, 'int...",Argentina Malbec,4.0,"Wow, this one was a surprise for being so youn...",2018-04-20T20:11:25.000Z,1.0
1,2017,1879,Malbec,Argentina,"{'acidity': 2.7203922, 'fizziness': None, 'int...",Argentina Malbec,4.0,For the price seems quite good. A little too o...,2019-04-01T03:24:42.000Z,1.0
2,2017,1879,Malbec,Argentina,"{'acidity': 2.7203922, 'fizziness': None, 'int...",Argentina Malbec,4.0,Lots of oak with dry qualities like you’d expe...,2019-08-10T20:43:25.000Z,1.0
3,2017,1879,Malbec,Argentina,"{'acidity': 2.7203922, 'fizziness': None, 'int...",Argentina Malbec,4.0,"Fantastic Malbec, can’t really go wrong here \...",2019-02-03T01:20:48.000Z,1.0
4,2017,1879,Malbec,Argentina,"{'acidity': 2.7203922, 'fizziness': None, 'int...",Argentina Malbec,4.0,"Nice pemegranate color. Medium acidity, medium...",2019-05-30T15:45:16.000Z,1.0


In [92]:
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

def delete_emojis(text):
    return emoji_pattern.sub(r'', text) #deletes emojis

In [88]:
delete_emojis(vivino_reviews['Note'][41619])

'My second Etna Rosso, volcanic wine as my good friend KK calls it \nA bright wine, light body, crisp acidity, leather, smoke, palate ripe cherries, vanilla, dust, quite earthy. Finish was relatively short lived but delicious. '

In [93]:
## some cleaning and stats if needed (convert string back to dict or list of tuples)
vivino_reviews['Note'] = vivino_reviews['Note'].apply(lambda x: delete_emojis(x))

In [78]:
## vectorizer loading

with open('../input/wine-preferences/vmodel.txt', 'rb') as f:
    cv = pickle.load(f)

In [41]:
## model loading
xgb_model = xgb.Booster()
xgb_model.load_model('../input/wine-preferences/big_model.json')

In [73]:
# 41620
# 41619

vivino_reviews['Note'][41619]

'My second Etna Rosso, volcanic wine as my good friend KK calls it 🗻\nA bright wine, light body, crisp acidity, leather, smoke, palate ripe cherries, vanilla, dust, quite earthy. Finish was relatively short lived but delicious. '

In [94]:
vivino_x = cv.transform(vivino_reviews['Note'])
xgb_vivino = xgb.DMatrix(vivino_x)

In [95]:
## sentiment prediction

sentiment_y = xgb_model.predict(xgb_vivino)
vivino_reviews['sentiment'] = np.where(np.array(sentiment_y) > 0.5, 1, 0)

In [96]:
vivino_reviews.head()

,Year,Wine ID,Wine Name,Country,Structure,Style,User Rating,Note,CreatedAt,Wine Type,sentiment
0,2017,1879,Malbec,Argentina,"{'acidity': 2.7203922, 'fizziness': None, 'int...",Argentina Malbec,4.0,"Wow, this one was a surprise for being so youn...",2018-04-20T20:11:25.000Z,1.0,1
1,2017,1879,Malbec,Argentina,"{'acidity': 2.7203922, 'fizziness': None, 'int...",Argentina Malbec,4.0,For the price seems quite good. A little too o...,2019-04-01T03:24:42.000Z,1.0,1
2,2017,1879,Malbec,Argentina,"{'acidity': 2.7203922, 'fizziness': None, 'int...",Argentina Malbec,4.0,Lots of oak with dry qualities like you’d expe...,2019-08-10T20:43:25.000Z,1.0,0
3,2017,1879,Malbec,Argentina,"{'acidity': 2.7203922, 'fizziness': None, 'int...",Argentina Malbec,4.0,"Fantastic Malbec, can’t really go wrong here \...",2019-02-03T01:20:48.000Z,1.0,1
4,2017,1879,Malbec,Argentina,"{'acidity': 2.7203922, 'fizziness': None, 'int...",Argentina Malbec,4.0,"Nice pemegranate color. Medium acidity, medium...",2019-05-30T15:45:16.000Z,1.0,1


In [97]:
len(vivino_reviews)

153172

In [99]:
vivino_reviews[vivino_reviews['sentiment']==1]['sentiment'].count()

90777

In [100]:
vivino_reviews.to_csv('vivino_sentiment1_100.csv')